In [1]:
import os
import sys
spark_home = os.environ.get('SPARK_HOME', None)
if not spark_home:
    raise ValueError('SPARK_HOME environment variable is not set')
sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, '/opt/spark/dimos-spark/python/lib/py4j-0.8.2.1-src.zip')) 
print spark_home
execfile(os.path.join(spark_home, 'python/pyspark/shell.py'))
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-csv_2.10:1.3.0 pyspark-shell'

#from pyspark import SparkContext

/opt/spark/current
Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 1.5.0
      /_/

Using Python version 2.7.12 (default, Jul  2 2016 17:42:40)
SparkContext available as sc, SQLContext available as sqlContext.


In [38]:
from datetime import datetime

In [56]:
#headers
'''
0 VendorID,
1 tpep_pickup_datetime,
2 tpep_dropoff_datetime,
3 passenger_count,
4 trip_distance,
5 pickup_longitude,
6 pickup_latitude,
7 RatecodeID,
8 store_and_fwd_flag,
9 dropoff_longitude,
10 dropoff_latitude,
11 payment_type,
12 fare_amount,
13 extra,
14 mta_tax,
15 tip_amount,
16 tolls_amount,
17 improvement_surcharge,
18 total_amount

'''
raw = sc.textFile('yellow_tripdata_2016-06.csv').map(lambda line:line.split(',')).map(lambda l:(l[1],l[2],l[6],l[5],l[10],l[9])).zipWithIndex().filter (lambda (k,v):v > 0)


In [73]:
rdd_tp_td_latp_lonp_latd_lond = raw.map(lambda (k,v):(datetime.strptime(str(k[0]), '%Y-%m-%d %H:%M:%S'),datetime.strptime(str(k[1]), '%Y-%m-%d %H:%M:%S'),float(str(k[2][:9])),float(str(k[3][:10])),float(str(k[4][:9])),float(str(k[5][:10]))))

In [86]:
rdd_plonglat_dlonglat = raw.map(lambda (k,v):((float(str(k[2][:9])),float(str(k[3][:10]))),(float(str(k[4][:9])),float(str(k[5][:10])))))

In [90]:
#raw.take(1)
#rdd_tp_td_latp_lonp_latd_lond.count()
#rdd_tp_td_latp_lonp_latd_lond.take(1)
rdd_plonglat_dlonglat.take(1)

[((40.760936, -73.98336), (40.753978, -73.977462))]

In [ ]:
'''
list the drop off nodes with degree less or equal to 10 
'''

In [88]:
rdd_ddgree = rdd_plonglat_dlonglat.map(lambda (x,y):(y,1)).reduceByKey(lambda x,y:x+y)

In [95]:
rdd_dlesseq10 = rdd_ddgree.filter(lambda (x,y): y<=10).map(lambda (x,y):(x,1))

In [96]:
tmp = rdd_plonglat_dlonglat.map(lambda (x,y):(y,x)).leftOuterJoin(rdd_dlesseq10)

In [98]:
filter_1 = tmp.filter(lambda (d,(p,r)): r is None).map(lambda (d,(p,r)):(p,d))

In [99]:
filter_1.count()

192667

In [100]:
rdd_pdgree = filter_1.map(lambda (p,d):(p,1)).reduceByKey(lambda x,y:x+y)

In [104]:
rdd_pdgree.sortBy(lambda (x,y):-y).take(10)

[((0.0, 0.0), 112602),
 ((40.750396, -73.935615), 56),
 ((40.64178, -74.08995), 54),
 ((40.754348, -73.99559), 51),
 ((40.73851, -73.999588), 50),
 ((40.298828, -73.996421), 50),
 ((40.743831, -73.976722), 48),
 ((40.746021, -73.933952), 47),
 ((40.746143, -73.97776), 45),
 ((40.730758, -74.001602), 45)]

'''
Pick-up, dropoff degree count
'''

In [75]:
rdd_pdgree = rdd_tp_td_latp_lonp_latd_lond.map(lambda x:((x[2],x[3]),1)).reduceByKey(lambda x,y:x+y)

In [80]:
#rdd_pdgree.sortBy(lambda (x,y):-y).take(10)
rdd_pdgree.filter(lambda (x,y):y > 1 and x != (0.0,0.0)).count()

1653214

In [77]:
rdd_ddgree = rdd_tp_td_latp_lonp_latd_lond.map(lambda x:((x[4],x[5]),1)).reduceByKey(lambda x,y:x+y)

In [78]:
rdd_ddgree.sortBy(lambda (x,y):-y).take(10)

[((0.0, 0.0), 127824),
 ((40.298828, -73.996421), 56),
 ((40.750396, -73.935615), 56),
 ((40.64178, -74.08995), 54),
 ((40.73851, -73.999588), 52),
 ((40.754348, -73.99559), 51),
 ((40.743831, -73.976722), 48),
 ((40.746021, -73.933952), 47),
 ((40.76062, -74.002731), 46),
 ((40.746143, -73.97776), 46)]

'''
chk if there is a problem converting the data
'''
def chktime(s):
    try:
        date_object = datetime.strptime(str(s), '%Y-%m-%d %H:%M:%S')
        return False
    except ValueError:
        return True
def chkstring(s):
    try:
        s = str(s)
        return False
    except ValueError:
        return True
def chkfloat(f):
    try:
        f = float(f)
        return False
    except ValueError:
        return True

tmp = raw.map(lambda (k,v):k[1]).filter(lambda s:chktime(s)==True)
tmp.collect()